In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
taiwan_0050 = pd.Series(['0050.TW'], index=[0])
taiwan_weighted = pd.Series(['^TWII'], index=[0])
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concat全部股票代號
stock_num = pd.concat([taiwan_weighted, taiwan_0050, stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(stock_num)
# print(type(allstock_info['ID'].values[0]))


In [3]:
# 刪除資料函式
def delete_data(stock_id, time = 0, dropindex = None):
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"   
    try:
        if  os.path.isfile(address):
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            if time in df_new.index:
                df_new = df_new.drop(time)
                df_new.to_csv(address,encoding='utf-8-sig')
                print("已刪除指定資料")
            elif time == 0:
                df_new = df_new.drop(df_new.index[dropindex])
                df_new.to_csv(address,encoding='utf-8-sig')
                print("已刪除指定資料")
            else:
                print('無此日期')
        else:
            print("無此檔案")
        print(stock_id, 'successful')
    except Exception as err:
        print(err)
        print(stock_id, '更新失敗')

In [ ]:
## 刪除資料
deletetime = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=-4)) , '%Y-%m-%d' )
print(deletetime)
stock_id = ['1210.TW']
for i in stock_num:
    try:
        delete_data(i, time = 0, dropindex = -1)
    except Exception as e:
        print(e)

In [ ]:
## 刪除資料
delay = 1
deletetime = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=-delay)) , '%Y-%m-%d' )
stock_id = '1101.TW'
for i in stock_num:
    try:
        delete_data(i,deletetime)
    except Exception as e:
        print(e)

In [ ]:
## 第一次製做獨立的ERS MA csv
def rs_rate():
    files = glob.glob('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/*.csv')
    rs250_list = []
    rs50_list = []
    rs20_list = []
    for i, path in enumerate(files):    
        try:
            hisdata = pd.read_csv(path,parse_dates = ["Date"])
            rs = hisdata[['Date', 'ID', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA']]
            if i == 0:
                rs250_df = rs[['Date', 'ID', 'RS 250EMA']]
                rs50_df = rs[['Date', 'ID', 'RS 50EMA']]
                rs20_df = rs[['Date', 'ID', 'RS 20EMA']]
            else:
                rs250_df = pd.concat([rs250_df, rs[['Date', 'ID', 'RS 250EMA']]], ignore_index=True)
                rs50_df = pd.concat([rs50_df, rs[['Date', 'ID', 'RS 50EMA']]], ignore_index=True)
                rs20_df = pd.concat([rs20_df, rs[['Date', 'ID', 'RS 20EMA']]], ignore_index=True)
        except Exception as e:
            print(e)
    rs250_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA250.csv')
    rs50_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA50.csv')
    rs20_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA20.csv')
rs_rate()

In [ ]:
## 第一次建立ERS rate的csv
files = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA250.csv')
dates = list(set(files['Date'].values))
files.set_index('Date', inplace=True)
files.dropna(inplace=True)
files['RS EMA250rate'] = 0
files2 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA50.csv')
files2.set_index('Date', inplace=True)
files2.dropna(inplace=True)
files2['RS EMA50rate'] = 0
files3 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA20.csv')
files3.set_index('Date', inplace=True)
files3.dropna(inplace=True)
files3['RS EMA20rate'] = 0
scaler = MinMaxScaler(feature_range=(0, 100))
length = len(dates)
for i, date in enumerate(dates):
    try:
        # print(len(files.loc[date, 'RS EMA250rate']))
        files.loc[date, 'RS EMA250rate'] = files.loc[date, 'RS 250EMA'].rank(method='dense', ascending=True)
        # print('排序完成')
        files.loc[date, 'RS EMA250rate'] = scaler.fit_transform(np.array(files.loc[date, 'RS EMA250rate']).reshape(-1, 1))
        print(date, '250EMA已完成', end=' | ', sep='  ')
    except Exception as e:
        # print(e)
        print(date, '250EMA失敗', end=' | ', sep='  ')
    try:
        # print(len(files2.loc[date, 'RS EMA50rate']))
        files2.loc[date, 'RS EMA50rate'] = files2.loc[date, 'RS 50EMA'].rank(method='dense', ascending=True)
        # print('排序完成')
        files2.loc[date, 'RS EMA50rate'] = scaler.fit_transform(np.array(files2.loc[date, 'RS EMA50rate']).reshape(-1, 1))
        print(date, '50EMA已完成', end=' | ', sep='  ')
    except Exception as e:
        # print(e)
        print(date, '50EMA失敗', end=' | ', sep='  ')
    try:
        # print(len(files3.loc[date, 'RS EMA20rate']))
        files3.loc[date, 'RS EMA20rate'] = files3.loc[date, 'RS 20EMA'].rank(method='dense', ascending=True)
        # print('排序完成')
        files3.loc[date, 'RS EMA20rate'] = scaler.fit_transform(np.array(files3.loc[date, 'RS EMA20rate']).reshape(-1, 1))
        print(date, '20EMA已完成', end=' | ', sep='  ')
    except Exception as e:
        # print(e)
        print(date, '20EMA失敗', end=' | ', sep='  ')
    print(f'{i}/{length}')
files.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA250rate.csv')
files2.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA50rate.csv')
files3.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA20rate.csv')

In [ ]:
## 第一次匯入所有歷史ERS rate
address = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA250rate.csv', index_col = "Date",parse_dates = ["Date"])
address2 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA50rate.csv', index_col = "Date",parse_dates = ["Date"])
address3 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsEMA20rate.csv', index_col = "Date",parse_dates = ["Date"])
stock_length = len(stock_num)
for n, i in enumerate(stock_num):
    print(i, end=' | ')
    try:
        stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', index_col = "Date",parse_dates = ["Date"])
        date = address.loc[address['ID'] == int(i.split('.')[0])].index.values
        rs = address.loc[address['ID'] == int(i.split('.')[0])].loc[date, 'RS EMA250rate']
        stock['RS EMA250rate'] = None
        stock.loc[date, 'RS EMA250rate'] = rs
        stock.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv')
        print('250EMA已完成', end=' | ')
    except Exception as e:
        # print(e)
        print('250EMA更新失敗', end=' | ')
    try:
        stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', index_col = "Date",parse_dates = ["Date"])
        date = address2.loc[address2['ID'] == int(i.split('.')[0])].index.values
        rs = address2.loc[address2['ID'] == int(i.split('.')[0])].loc[date, 'RS EMA50rate']
        stock['RS EMA50rate'] = None
        stock.loc[date, 'RS EMA50rate'] = rs
        stock.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv')
        print('50EMA已完成', end=' | ')
    except Exception as e:
        # print(e)
        print('50EMA更新失敗', end=' | ')
    try:
        stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', index_col = "Date",parse_dates = ["Date"])
        date = address3.loc[address3['ID'] == int(i.split('.')[0])].index.values
        rs = address3.loc[address3['ID'] == int(i.split('.')[0])].loc[date, 'RS EMA20rate']
        stock['RS EMA20rate'] = None
        stock.loc[date, 'RS EMA20rate'] = rs
        stock.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv')
        print('20EMA已完成', end=' | ')
    except Exception as e:
        # print(e)
        print('20EMA更新失敗', end=' | ')
    print(f'{n}/{stock_length}')

In [ ]:
## 第一次製做獨立的RS MA csv
def rs_rate():
    files = glob.glob('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/*.csv')
    rs250_list = []
    rs50_list = []
    rs20_list = []
    for i, path in enumerate(files):    
        try:
            hisdata = pd.read_csv(path,parse_dates = ["Date"])
            rs = hisdata[['Date', 'ID', 'RS 250MA', 'RS 50MA', 'RS 20MA']]
            if i == 0:
                rs250_df = rs[['Date', 'ID', 'RS 250MA']]
                rs50_df = rs[['Date', 'ID', 'RS 50MA']]
                rs20_df = rs[['Date', 'ID', 'RS 20MA']]
            else:
                rs250_df = pd.concat([rs250_df, rs[['Date', 'ID', 'RS 250MA']]], ignore_index=True)
                rs50_df = pd.concat([rs50_df, rs[['Date', 'ID', 'RS 50MA']]], ignore_index=True)
                rs20_df = pd.concat([rs20_df, rs[['Date', 'ID', 'RS 20MA']]], ignore_index=True)
        except Exception as e:
            print(e)
    rs250_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA250.csv')
    rs50_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA50.csv')
    rs20_df.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA20.csv')
rs_rate()

In [ ]:
## 第一次建立RS rate的csv
files = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA250.csv')
dates = list(set(files['Date'].values))
files.set_index('Date', inplace=True)
files.dropna(inplace=True)
files['RS MA250rate'] = 0
files2 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA50.csv')
files2.set_index('Date', inplace=True)
files2.dropna(inplace=True)
files2['RS MA50rate'] = 0
files3 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA20.csv')
files3.set_index('Date', inplace=True)
files3.dropna(inplace=True)
files3['RS MA20rate'] = 0
scaler = MinMaxScaler(feature_range=(0, 100))
length = len(dates)
for i, date in enumerate(dates):
    try:
        # print(len(files.loc[date, 'RS MA250rate']))
        files.loc[date, 'RS MA250rate'] = files.loc[date, 'RS 250MA'].rank(method='dense', ascending=True)
        # print('排序完成')
        files.loc[date, 'RS MA250rate'] = scaler.fit_transform(np.array(files.loc[date, 'RS MA250rate']).reshape(-1, 1))
        print(date, '250MA已完成', end=' | ', sep='  ')
    except Exception as e:
        # print(e)
        print(date, '250MA失敗', end=' | ', sep='  ')
    try:
        # print(len(files2.loc[date, 'RS MA50rate']))
        files2.loc[date, 'RS MA50rate'] = files2.loc[date, 'RS 50MA'].rank(method='dense', ascending=True)
        # print('排序完成')
        files2.loc[date, 'RS MA50rate'] = scaler.fit_transform(np.array(files2.loc[date, 'RS MA50rate']).reshape(-1, 1))
        print(date, '50MA已完成', end=' | ', sep='  ')
    except Exception as e:
        # print(e)
        print(date, '50MA失敗', end=' | ', sep='  ')
    try:
        # print(len(files3.loc[date, 'RS MA20rate']))
        files3.loc[date, 'RS MA20rate'] = files3.loc[date, 'RS 20MA'].rank(method='dense', ascending=True)
        # print('排序完成')
        files3.loc[date, 'RS MA20rate'] = scaler.fit_transform(np.array(files3.loc[date, 'RS MA20rate']).reshape(-1, 1))
        print(date, '20MA已完成', end=' | ', sep='  ')
    except Exception as e:
        # print(e)
        print(date, '20MA失敗', end=' | ', sep='  ')
    print(f'{i}/{length}')
files.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA250rate.csv')
files2.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA50rate.csv')
files3.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA20rate.csv')

In [ ]:
## 第一次匯入所有歷史RS rate
address = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA250rate.csv', index_col = "Date",parse_dates = ["Date"])
address2 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA50rate.csv', index_col = "Date",parse_dates = ["Date"])
address3 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/rsMA20rate.csv', index_col = "Date",parse_dates = ["Date"])
stock_length = len(stock_num)
for n, i in enumerate(stock_num):
    print(i, end=' | ')
    try:
        stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', index_col = "Date",parse_dates = ["Date"])
        date = address.loc[address['ID'] == int(i.split('.')[0])].index.values
        rs = address.loc[address['ID'] == int(i.split('.')[0])].loc[date, 'RS MA250rate']
        stock['RS MA250rate'] = None
        stock.loc[date, 'RS MA250rate'] = rs
        stock.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', encoding='utf-8-sig')
        print('250MA已完成', end=' | ')
    except Exception as e:
        # print(e)
        print('250MA更新失敗', end=' | ')
    try:
        stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', index_col = "Date",parse_dates = ["Date"])
        date = address2.loc[address2['ID'] == int(i.split('.')[0])].index.values
        rs = address2.loc[address2['ID'] == int(i.split('.')[0])].loc[date, 'RS MA50rate']
        stock['RS MA50rate'] = None
        stock.loc[date, 'RS MA50rate'] = rs
        stock.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', encoding='utf-8-sig')
        print('50MA已完成', end=' | ')
    except Exception as e:
        # print(e)
        print('50MA更新失敗', end=' | ')
    try:
        stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', index_col = "Date",parse_dates = ["Date"])
        date = address3.loc[address3['ID'] == int(i.split('.')[0])].index.values
        rs = address3.loc[address3['ID'] == int(i.split('.')[0])].loc[date, 'RS MA20rate']
        stock['RS MA20rate'] = None
        stock.loc[date, 'RS MA20rate'] = rs
        stock.to_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(i) + '.csv', encoding='utf-8-sig')
        print('20MA已完成', end=' | ')
    except Exception as e:
        # print(e)
        print('20MA更新失敗', end=' | ')
    print(f'{n}/{stock_length}')

In [8]:
##讀寫成csv檔
def stock_data(stock_id,time_start,time_end) :
    days = 24 * 60 * 60    #一天有86400秒 
    initial = datetime.datetime.strptime( '1970-01-01' , '%Y-%m-%d' )
    start = datetime.datetime.strptime( time_start , '%Y-%m-%d' )
    end = datetime.datetime.strptime( time_end, '%Y-%m-%d' )
    period1 = start - initial
    period2 = end - initial
    s1 = period1.days * days
    s2 = period2.days * days
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=" + str(s1) + "&period2=" + str(s2) + "&interval=1d&events=history&includeAdjustedClose=true"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}
    response = requests.get(url,headers = headers)
    df = pd.read_csv(StringIO(response.text),index_col = "Date",parse_dates = ["Date"])
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"

    try:
        if  os.path.isfile(address):
            
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            if time_start not in df_new.index:
                df_new = pd.concat([df_new,df])
                df_new.to_csv(address,encoding='utf-8')
                # print("已更新到最新資料")
            else:
                print(stock_id, "已是最新價格資料，無需更新")
        else:
            df = df.dropna()
            df.to_csv(address,encoding='utf-8')
            print("此為新資料，已創建csv檔")
    except Exception as err:
        print(err)
        print('價格資料更新失敗')
        

In [24]:
# 指標更新函式
def indicator(stock_id):
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"   
    try:
        if  os.path.isfile(address):
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            try:
                df_new['ID'] = stock_id.split('.')[0]
            except Exception as e:
                print(e)
                print(stock_id,1)
            df_new['5MA'] = talib.SMA(df_new["Adj Close"], 5)
            df_new['10MA'] = talib.SMA(df_new["Adj Close"], 10)
            df_new['20MA'] = talib.SMA(df_new["Adj Close"], 20)
            df_new['50MA'] = talib.SMA(df_new["Adj Close"], 50)
            df_new['100MA'] = talib.SMA(df_new["Adj Close"], 100)
            df_new['150MA'] = talib.SMA(df_new["Adj Close"], 150)
            df_new['200MA'] = talib.SMA(df_new["Adj Close"], 200)  
            df_new['200MA ROCP'] = np.array(map(lambda x : round(x*100,2), talib.ROCP(df_new["200MA"], timeperiod=1)))
            df_new['200MA ROCP 20MA'] = talib.SMA(df_new["200MA ROCP"], 20)
            df_new['200MA ROCP 60MA'] = talib.SMA(df_new["200MA ROCP"], 60)
            df_new['5Max'] = talib.MAX(df_new['High'], 5)
            df_new['250Max'] = talib.MAX(df_new['High'], 250)
            df_new['5Min'] = talib.MIN(df_new['Low'], 5)
            df_new['250Min'] = talib.MIN(df_new['Low'], 250)
            df_new['Volume 50MA'] = talib.SMA(df_new['Volume'], 50)
            df_new['ATR250'] = talib.ATR(high=df_new['High'], low=df_new["Low"], close=df_new["Adj Close"], timeperiod=250)
            df_new['ATR50'] = talib.ATR(high=df_new['High'], low=df_new["Low"], close=df_new["Adj Close"], timeperiod=50)
            df_new['ATR20'] = talib.ATR(high=df_new['High'], low=df_new["Low"], close=df_new["Adj Close"], timeperiod=20)   
            # 漲幅
            df_new['ROCP'] = np.array(map(lambda x : round(x*100,2), talib.ROCP(df_new["Adj Close"], timeperiod=1)))
            df_new['OBV'] = talib.OBV(df_new["Adj Close"], df_new["Volume"])
            df_new['beta'] = talib.BETA(df_new['High'].values, df_new['Low'].values, timeperiod = 50)
            stockindex = df_new.index.values
            index = []
            if stock_id not in ['^TWII', '0050.TW']:
                df_new['產業別'] = allstock_info.loc[df_new['ID'].values[0], '產業別']
            comparestock = pd.read_csv(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + '^TWII' + ".csv",index_col = "Date",parse_dates = ["Date"])
            compareindex = comparestock.index.values
            
            for i in stockindex:
                if i in compareindex:
                    index.append(i)
            df_new['RS value'] = 0
            df_new.loc[index, 'RS value'] = list(map(lambda stock,compare: stock-compare, df_new.loc[index, 'ROCP'], comparestock.loc[index, 'ROCP']))
            df_new['RS 250MA'] = np.array(talib.SMA(df_new['RS value'], 250))*100
            df_new['RS 50MA'] = np.array(talib.SMA(df_new['RS value'], 50))*100
            df_new['RS 20MA'] = np.array(talib.SMA(df_new['RS value'], 20))*100
            df_new['RS 250EMA'] = np.array(talib.EMA(df_new['RS value'], 250))*100
            df_new['RS 50EMA'] = np.array(talib.EMA(df_new['RS value'], 50))*100
            df_new['RS 20EMA'] = np.array(talib.EMA(df_new['RS value'], 20))*100
            # df_new['RS 250MA 10MAX'] = talib.MAX(df_new['RS 250MA'], 10)
            # df_new['RS 50MA 10MAX'] = talib.MAX(df_new['RS 50MA'], 10)
            # df_new['RS 20MA 10MAX'] = talib.MAX(df_new['RS 20MA'], 10)
            # df_new['RS 250EMA 10MAX'] = talib.MAX(df_new['RS 250EMA'], 10)
            # df_new['RS 50EMA 10MAX'] = talib.MAX(df_new['RS 50EMA'], 10)
            # df_new['RS 20EMA 10MAX'] = talib.MAX(df_new['RS 20EMA'], 10)
            df_new.to_csv(address,encoding='utf-8-sig')
            # print("指標已更新到最新資料")
    except Exception as err:
        print(err)
        print(stock_id + '指標更新失敗')

In [ ]:
# 第一次建立csv檔
time_start = "2000-01-01"
time_end = "2023-10-02"
for i in stock_num:   # ['1210.TW']
    try:
        stock_data(i, time_start,time_end)
        print(i + "successful")
    except Exception as e:
        print(e)
        print(i + "fail")

In [25]:
# 每日更新csv
delay = 0
today=datetime.date.today()+ datetime.timedelta(days=-delay)
tomorrow = today + datetime.timedelta(days=1)
time_start = str(today)
time_end = str(tomorrow)
for i in stock_num :   
    # 更新價量
    try:
        stock_data(i, time_start,time_end)
        # print(i + "successful")
    except Exception as e:
        try:
            stock_data(i, time_start,time_end)
            # print(i + "successful")
        except Exception as e:
            print(i + "fail")
            print(e)
    # 更新指標
    
    try:
        indicator(i)
        # print(i + "successful")
    except Exception as e:
        try:
            indicator(i)
            # print(i + "successful")
        except Exception as e:
            print(i + "fail")
            print(e)


1210.TW 已是最新價格資料，無需更新
1236.TWfail
Missing column provided to 'parse_dates': 'Date'
1538.TWfail
Missing column provided to 'parse_dates': 'Date'
1776.TWfail
Missing column provided to 'parse_dates': 'Date'
1805.TWfail
Missing column provided to 'parse_dates': 'Date'
inputs are all NaN
2025.TW指標更新失敗
inputs are all NaN
2317.TW指標更新失敗
2443.TWfail
Missing column provided to 'parse_dates': 'Date'
2489.TWfail
Missing column provided to 'parse_dates': 'Date'
inputs are all NaN
4569.TW指標更新失敗
4994.TWfail
Missing column provided to 'parse_dates': 'Date'
6225.TWfail
Missing column provided to 'parse_dates': 'Date'
inputs are all NaN
6657.TW指標更新失敗
inputs are all NaN
6863.TW指標更新失敗
inputs are all NaN
6901.TW指標更新失敗
2924.TWOfail
Missing column provided to 'parse_dates': 'Date'
3085.TWOfail
Missing column provided to 'parse_dates': 'Date'
3089.TWOfail
Missing column provided to 'parse_dates': 'Date'
3115.TWOfail
Missing column provided to 'parse_dates': 'Date'
3226.TWOfail
Missing column provided to 'par

In [51]:
#每日更新RS參考資料
def update_rs_rate(timestart):
    rs20rate = []
    first = 1
    rs_df = 0
    for i, id in enumerate(stock_num):
        try:
        
            stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(id) + '.csv',parse_dates = ["Date"])
            stock.set_index('Date', inplace = True)
            if timestart in stock.index:    
                if first:
                    rs_df = pd.DataFrame(stock.loc[timestart, ['ID', 'RS 250MA', 'RS 50MA', 'RS 20MA', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA']]).transpose()
                    print(rs_df)
                    rs_df.set_index('ID', inplace = True)
                    first = 0
                else:
                    stock = pd.DataFrame(stock.loc[timestart, ['ID', 'RS 250MA', 'RS 50MA', 'RS 20MA', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA']]).transpose()
                    stock.set_index('ID', inplace = True)
                    rs_df = pd.concat([rs_df, stock])
            # print(id, '更新RS參考資料完成')
        except Exception as e:
            print(e)
            print(id, '更新RS參考資料失敗')
    return rs_df


In [40]:
# 每日做RSrate參考資料
def rs_rate_dataset(rs_df):
    rs_df['RS 250rate'] = 0
    rs_df['RS 50rate'] = 0
    rs_df['RS 20rate'] = 0
    rs_df['RS EMA250rate'] = 0
    rs_df['RS EMA50rate'] = 0
    rs_df['RS EMA20rate'] = 0
    scaler = MinMaxScaler(feature_range=(0, 100))
    try:
        rs_df['RS 250rate'] = rs_df['RS 250MA'].rank(method='dense', ascending=True)
        print('RSrate參考資料250排序完成')
        rs_df['RS 250rate'] = scaler.fit_transform(np.array(rs_df['RS 250rate']).reshape(-1, 1))
        print('RSrate參考資料250已完成')
    except Exception as e:
        print(e)
        print('RSrate參考資料250失敗')
    try:
        rs_df['RS 50rate'] = rs_df['RS 50MA'].rank(method='dense', ascending=True)
        print('RSrate參考資料50排序完成')
        rs_df['RS 50rate'] = scaler.fit_transform(np.array(rs_df['RS 50rate']).reshape(-1, 1))
        print('RSrate參考資料50已完成')
    except Exception as e:
        print(e)
        print('RSrate參考資料50失敗')
    try:
        rs_df['RS 20rate'] = rs_df['RS 20MA'].rank(method='dense', ascending=True)
        print('RSrate參考資料20排序完成')
        rs_df['RS 20rate'] = scaler.fit_transform(np.array(rs_df['RS 20rate']).reshape(-1, 1))
        print('RSrate參考資料20已完成')
    except Exception as e:
        print(e)
        print('RSrate參考資料20失敗')
    try:
        rs_df['RS EMA250rate'] = rs_df['RS 250EMA'].rank(method='dense', ascending=True)
        print('RSEMArate參考資料250排序完成')
        rs_df['RS EMA250rate'] = scaler.fit_transform(np.array(rs_df['RS EMA250rate']).reshape(-1, 1))
        print('RSEMArate參考資料250已完成')
    except Exception as e:
        print(e)
        print('RSEMArate參考資料250失敗')
    try:
        rs_df['RS EMA50rate'] = rs_df['RS 50EMA'].rank(method='dense', ascending=True)
        print('RSEMArate參考資料50排序完成')
        rs_df['RS EMA50rate'] = scaler.fit_transform(np.array(rs_df['RS EMA50rate']).reshape(-1, 1))
        print('RSEMArate參考資料50已完成')
    except Exception as e:
        print(e)
        print('RSEMArate參考資料50失敗')
    try:
        rs_df['RS EMA20rate'] = rs_df['RS 20EMA'].rank(method='dense', ascending=True)
        print('RSEMArate參考資料20排序完成')
        rs_df['RS EMA20rate'] = scaler.fit_transform(np.array(rs_df['RS EMA20rate']).reshape(-1, 1))
        print('RSEMArate參考資料20已完成')
    except Exception as e:
        print(e)
        print('RSEMArate參考資料20失敗')
    return rs_df


In [41]:
# 每日更新RS rate到每個csv
def daily_RS_update(timestart, rs_df):
    for i, id in enumerate(stock_num):
        name = id
        address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(id) + '.csv'
        try:
            stock = pd.read_csv(address, index_col = "Date",parse_dates = ["Date"])
        except Exception as e:
            print(e)
            print(name, 'RS rate更新失敗')
        try:
            id = int(id.split('.')[0])
            if timestart in stock.index:
                stock.loc[timestart, ['RS 20rate', 'RS 50rate', 'RS 250rate', 'RS EMA20rate', 'RS EMA50rate', 'RS EMA250rate']] = rs_df.loc[id, ['RS 20rate', 'RS 50rate', 'RS 250rate', 'RS EMA20rate', 'RS EMA50rate', 'RS EMA250rate']]
                stock.to_csv(address,encoding='utf-8-sig')
                print(name, 'RS rate更新成功')
            else:
                print(name, '沒有RS資料')
        except Exception as e:
            print(e)
            print(name, 'RS rate更新失敗')

In [ ]:
# 準備RS rate資料
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")
delay = 0
timestart = datetime.datetime.strptime(str(datetime.date.today()+ datetime.timedelta(days=-delay)) , '%Y-%m-%d' )
rs_df = update_rs_rate(timestart)
print('rs rate更新完成')
rs_df = rs_rate_dataset(rs_df)
rs_df.head(10)

In [ ]:
# 將RS rate資料匯入每個csv
daily_RS_update(timestart, rs_df)

In [53]:
def update_RS_MAX(stock_id):
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"   
    try:
        if  os.path.isfile(address):
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            try:
                df_new['ID'] = stock_id.split('.')[0]
            except Exception as e:
                print(e)
                print(stock_id,1)
            # 10 max
            df_new['RS 250MA 10MAX'] = talib.MAX(df_new['RS 250rate'], 10)
            df_new['RS 50MA 10MAX'] = talib.MAX(df_new['RS 50rate'], 10)
            df_new['RS 20MA 10MAX'] = talib.MAX(df_new['RS 20rate'], 10)
            df_new['RS 250MA is 10MAX'] = df_new['RS 250rate'].round(1)>=df_new['RS 250MA 10MAX'].round(1)
            df_new['RS 50MA is 10MAX'] = df_new['RS 50rate'].round(1)>=df_new['RS 50MA 10MAX'].round(1)
            df_new['RS 20MA is 10MAX'] = df_new['RS 20rate'].round(1)>=df_new['RS 20MA 10MAX'].round(1)
            df_new['RS 250EMA 10MAX'] = talib.MAX(df_new['RS EMA250rate'], 10)
            df_new['RS 50EMA 10MAX'] = talib.MAX(df_new['RS EMA50rate'], 10)
            df_new['RS 20EMA 10MAX'] = talib.MAX(df_new['RS EMA20rate'], 10)
            df_new['RS 250EMA is 10MAX'] = df_new['RS EMA250rate'].round(1)>=df_new['RS 250EMA 10MAX'].round(1)
            df_new['RS 50EMA is 10MAX'] = df_new['RS EMA50rate'].round(1)>=df_new['RS 50EMA 10MAX'].round(1)
            df_new['RS 20EMA is 10MAX'] = df_new['RS EMA20rate'].round(1)>=df_new['RS 20EMA 10MAX'].round(1)
            # 20 max
            df_new['RS 250MA 20MAX'] = talib.MAX(df_new['RS 250rate'], 20)
            df_new['RS 50MA 20MAX'] = talib.MAX(df_new['RS 50rate'], 20)
            df_new['RS 20MA 20MAX'] = talib.MAX(df_new['RS 20rate'], 20)
            df_new['RS 250MA is 20MAX'] = df_new['RS 250rate'].round(1)>=df_new['RS 250MA 20MAX'].round(1)
            df_new['RS 50MA is 20MAX'] = df_new['RS 50rate'].round(1)>=df_new['RS 50MA 20MAX'].round(1)
            df_new['RS 20MA is 20MAX'] = df_new['RS 20rate'].round(1)>=df_new['RS 20MA 20MAX'].round(1)
            df_new['RS 250EMA 20MAX'] = talib.MAX(df_new['RS EMA250rate'], 20)
            df_new['RS 50EMA 20MAX'] = talib.MAX(df_new['RS EMA50rate'], 20)
            df_new['RS 20EMA 20MAX'] = talib.MAX(df_new['RS EMA20rate'], 20)
            df_new['RS 250EMA is 20MAX'] = df_new['RS EMA250rate'].round(1)>=df_new['RS 250EMA 20MAX'].round(1)
            df_new['RS 50EMA is 20MAX'] = df_new['RS EMA50rate'].round(1)>=df_new['RS 50EMA 20MAX'].round(1)
            df_new['RS 20EMA is 20MAX'] = df_new['RS EMA20rate'].round(1)>=df_new['RS 20EMA 20MAX'].round(1)
            # 50 max
            df_new['RS 250MA 50MAX'] = talib.MAX(df_new['RS 250rate'], 50)
            df_new['RS 50MA 50MAX'] = talib.MAX(df_new['RS 50rate'], 50)
            df_new['RS 20MA 50MAX'] = talib.MAX(df_new['RS 20rate'], 50)
            df_new['RS 250MA is 50MAX'] = df_new['RS 250rate'].round(1)>=df_new['RS 250MA 50MAX'].round(1)
            df_new['RS 50MA is 50MAX'] = df_new['RS 50rate'].round(1)>=df_new['RS 50MA 50MAX'].round(1)
            df_new['RS 20MA is 50MAX'] = df_new['RS 20rate'].round(1)>=df_new['RS 20MA 50MAX'].round(1)
            df_new['RS 250EMA 50MAX'] = talib.MAX(df_new['RS EMA250rate'], 50)
            df_new['RS 50EMA 50MAX'] = talib.MAX(df_new['RS EMA50rate'], 50)
            df_new['RS 20EMA 50MAX'] = talib.MAX(df_new['RS EMA20rate'], 50)
            df_new['RS 250EMA is 50MAX'] = df_new['RS EMA250rate'].round(1)>=df_new['RS 250EMA 50MAX'].round(1)
            df_new['RS 50EMA is 50MAX'] = df_new['RS EMA50rate'].round(1)>=df_new['RS 50EMA 50MAX'].round(1)
            df_new['RS 20EMA is 50MAX'] = df_new['RS EMA20rate'].round(1)>=df_new['RS 20EMA 50MAX'].round(1)
            # 250 max
            df_new['RS 250MA 250MAX'] = talib.MAX(df_new['RS 250rate'], 250)
            df_new['RS 50MA 250MAX'] = talib.MAX(df_new['RS 50rate'], 250)
            df_new['RS 20MA 250MAX'] = talib.MAX(df_new['RS 20rate'], 250)
            df_new['RS 250MA is 250MAX'] = df_new['RS 250rate'].round(1)>=df_new['RS 250MA 250MAX'].round(1)
            df_new['RS 50MA is 250MAX'] = df_new['RS 50rate'].round(1)>=df_new['RS 50MA 250MAX'].round(1)
            df_new['RS 20MA is 250MAX'] = df_new['RS 20rate'].round(1)>=df_new['RS 20MA 250MAX'].round(1)
            df_new['RS 250EMA 250MAX'] = talib.MAX(df_new['RS EMA250rate'], 250)
            df_new['RS 50EMA 250MAX'] = talib.MAX(df_new['RS EMA50rate'], 250)
            df_new['RS 20EMA 250MAX'] = talib.MAX(df_new['RS EMA20rate'], 250)
            df_new['RS 250EMA is 250MAX'] = df_new['RS EMA250rate'].round(1)>=df_new['RS 250EMA 250MAX'].round(1)
            df_new['RS 50EMA is 250MAX'] = df_new['RS EMA50rate'].round(1)>=df_new['RS 50EMA 250MAX'].round(1)
            df_new['RS 20EMA is 250MAX'] = df_new['RS EMA20rate'].round(1)>=df_new['RS 20EMA 250MAX'].round(1)
            df_new.to_csv(address,encoding='utf-8-sig')
            # print("指標已更新到最新資料")
    except Exception as err:
        print(err)
        print(stock_id + '指標更新失敗')

In [ ]:
# 更新columns name
for i in stock_num[2:] :   
    
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + i + ".csv"
    try:
        df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
        df_new.drop(['RS 20rate', 'RS 50rate', 'RS 250rate'], axis= 1, inplace=True)
        new_col = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ID', '5MA',
       '10MA', '20MA', '50MA', '100MA', '150MA', '200MA', '200MA ROCP',
       '200MA ROCP 20MA', '200MA ROCP 60MA', '5Max', '250Max', '5Min',
       '250Min', 'Volume 50MA', 'ROCP', 'OBV', 'beta', '產業別', 'RS value',
       'RS 250MA', 'RS 50MA', 'RS 20MA', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA',
       'RS 250MA 10MAX', 'RS 50MA 10MAX', 'RS 20MA 10MAX', 'RS 250EMA 10MAX',
       'RS 50EMA 10MAX', 'RS 20EMA 10MAX', 'RS EMA250rate', 'RS EMA50rate',
       'RS EMA20rate', 'RS 250rate', 'RS 50rate', 'RS 20rate']
        df_new.columns = new_col
        df_new.to_csv(address,encoding='utf-8-sig')
        # print(i + "successful")
    except Exception as e:
        print(i, e)


In [ ]:
for i in stock_num :   
    # 更新RS MAX
    try:
        update_RS_MAX(i)
        # print(i + "successful")
    except Exception as e:
        try:
            update_RS_MAX(i)
            # print(i + "successful")
        except Exception as e:
            print(i + "fail")
            print(e)


## data.py 測試

In [ ]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")
##讀寫成csv檔
def stock_data(stock_id,time_start,time_end) :
    days = 24 * 60 * 60    #一天有86400秒 
    initial = datetime.datetime.strptime( '1970-01-01' , '%Y-%m-%d' )
    start = datetime.datetime.strptime( time_start , '%Y-%m-%d' )
    end = datetime.datetime.strptime( time_end, '%Y-%m-%d' )
    period1 = start - initial
    period2 = end - initial
    s1 = period1.days * days
    s2 = period2.days * days
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=" + str(s1) + "&period2=" + str(s2) + "&interval=1d&events=history&includeAdjustedClose=true"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}
    response = requests.get(url,headers = headers)
    df = pd.read_csv(StringIO(response.text),index_col = "Date",parse_dates = ["Date"])
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"

    try:
        if  os.path.isfile(address):
            
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            if time_start not in df_new.index:
                df_new = pd.concat([df_new,df])
                df_new.to_csv(address,encoding='utf-8')
                # print("已更新到最新資料")
            else:
                print(stock_id, "已是最新價格資料，無需更新")
        else:
            df = df.dropna()
            df.to_csv(address,encoding='utf-8')
            print("此為新資料，已創建csv檔")
    except Exception as err:
        print(err)
        print('價格資料更新失敗')
# 指標更新函式
def indicator(stock_id):
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"   
    comparestock = pd.read_csv(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + '^TWII' + ".csv",index_col = "Date",parse_dates = ["Date"])
    compareindex = comparestock.index.values
    try:
        if  os.path.isfile(address):
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            try:
                df_new['ID'] = stock_id.split('.')[0]
            except Exception as e:
                print(e)
                print(stock_id)
            if stock_id not in ['^TWII', '0050.TW']:
                df_new['產業別'] = allstock_info.loc[df_new['ID'].values[0], '產業別']
            df_new['5MA'] = talib.SMA(df_new["Adj Close"], 5)
            df_new['10MA'] = talib.SMA(df_new["Adj Close"], 10)
            df_new['20MA'] = talib.SMA(df_new["Adj Close"], 20)
            df_new['50MA'] = talib.SMA(df_new["Adj Close"], 50)
            df_new['100MA'] = talib.SMA(df_new["Adj Close"], 100)
            df_new['150MA'] = talib.SMA(df_new["Adj Close"], 150)
            df_new['200MA'] = talib.SMA(df_new["Adj Close"], 200)
            df_new['200MA ROCP'] = np.array(map(lambda x : round(x*100,2), talib.ROCP(df_new["200MA"], timeperiod=1)))
            df_new['200MA ROCP 20MA'] = talib.SMA(df_new["200MA ROCP"], 20)
            df_new['200MA ROCP 60MA'] = talib.SMA(df_new["200MA ROCP"], 60)
            df_new['5Max'] = talib.MAX(df_new['High'], 5)
            df_new['250Max'] = talib.MAX(df_new['High'], 250)
            df_new['5Min'] = talib.MIN(df_new['Low'], 5)
            df_new['250Min'] = talib.MIN(df_new['Low'], 250)
            df_new['Volume 50MA'] = talib.SMA(df_new['Volume'], 50)
            # 漲幅
            df_new['ROCP'] = np.array(map(lambda x : round(x*100,2), talib.ROCP(df_new["Adj Close"], timeperiod=1)))
            df_new['OBV'] = talib.OBV(df_new["Adj Close"], df_new["Volume"])
            df_new['beta'] = talib.BETA(df_new['High'].values, df_new['Low'].values, timeperiod = 50)
            stockindex = df_new.index.values
            index = []
            for i in stockindex:
                if i in compareindex:
                    index.append(i)
            df_new['RS value'] = 0
            df_new.loc[index, 'RS value'] = list(map(lambda stock,compare: stock-compare, df_new.loc[index, 'ROCP'], comparestock.loc[index, 'ROCP']))
            df_new['RS 250MA'] = np.array(talib.SMA(df_new['RS value'], 250))*100
            df_new['RS 50MA'] = np.array(talib.SMA(df_new['RS value'], 50))*100
            df_new['RS 20MA'] = np.array(talib.SMA(df_new['RS value'], 20))*100
            df_new['RS 250EMA'] = np.array(talib.EMA(df_new['RS value'], 250))*100
            df_new['RS 50EMA'] = np.array(talib.EMA(df_new['RS value'], 50))*100
            df_new['RS 20EMA'] = np.array(talib.EMA(df_new['RS value'], 20))*100
            df_new.to_csv(address,encoding='utf-8-sig')
            # print("指標已更新到最新資料")
        else:
            pass
    except Exception as err:
        print(err)
        print(stock_id, '指標更新失敗')
#每日更新RS參考資料
def update_rs_rate(timestart):
    rs20rate = []
    first = 1
    rs_df = 0
    for i, id in enumerate(stock_num):
        try:
            stock = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(id) + '.csv',parse_dates = ["Date"])
            stock.set_index('Date', inplace = True)
            if timestart in stock.index:    
                if first:
                    rs_df = pd.DataFrame(stock.loc[timestart, ['ID', 'RS 250MA', 'RS 50MA', 'RS 20MA', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA']]).transpose()
                    print(rs_df)
                    rs_df.set_index('ID', inplace = True)
                    first = 0
                else:
                    stock = pd.DataFrame(stock.loc[timestart, ['ID', 'RS 250MA', 'RS 50MA', 'RS 20MA', 'RS 250EMA', 'RS 50EMA', 'RS 20EMA']]).transpose()
                    stock.set_index('ID', inplace = True)
                    rs_df = pd.concat([rs_df, stock])
            # print(id, '更新RS參考資料完成')
        except Exception as e:
            print(e)
            print(id, '更新RS參考資料失敗')
    return rs_df
# 每日做RSrate參考資料
def rs_rate_dataset(rs_df):
    rs_df['RS 250rate'] = 0
    rs_df['RS 50rate'] = 0
    rs_df['RS 20rate'] = 0
    rs_df['RS EMA250rate'] = 0
    rs_df['RS EMA50rate'] = 0
    rs_df['RS EMA20rate'] = 0
    scaler = MinMaxScaler(feature_range=(0, 100))
    try:
        rs_df['RS 250rate'] = rs_df['RS 250MA'].rank(method='dense', ascending=True)
        print('RSrate參考資料250排序完成')
        rs_df['RS 250rate'] = scaler.fit_transform(np.array(rs_df['RS 250rate']).reshape(-1, 1))
        print('RSrate參考資料250已完成')
    except Exception as e:
        print(e)
        print('RSrate參考資料250失敗')
    try:
        rs_df['RS 50rate'] = rs_df['RS 50MA'].rank(method='dense', ascending=True)
        print('RSrate參考資料50排序完成')
        rs_df['RS 50rate'] = scaler.fit_transform(np.array(rs_df['RS 50rate']).reshape(-1, 1))
        print('RSrate參考資料50已完成')
    except Exception as e:
        print(e)
        print('RSrate參考資料50失敗')
    try:
        rs_df['RS 20rate'] = rs_df['RS 20MA'].rank(method='dense', ascending=True)
        print('RSrate參考資料20排序完成')
        rs_df['RS 20rate'] = scaler.fit_transform(np.array(rs_df['RS 20rate']).reshape(-1, 1))
        print('RSrate參考資料20已完成')
    except Exception as e:
        print(e)
        print('RSrate參考資料20失敗')
    try:
        rs_df['RS EMA250rate'] = rs_df['RS 250EMA'].rank(method='dense', ascending=True)
        print('RSEMArate參考資料250排序完成')
        rs_df['RS EMA250rate'] = scaler.fit_transform(np.array(rs_df['RS EMA250rate']).reshape(-1, 1))
        print('RSEMArate參考資料250已完成')
    except Exception as e:
        print(e)
        print('RSEMArate參考資料250失敗')
    try:
        rs_df['RS EMA50rate'] = rs_df['RS EMA50rate'].rank(method='dense', ascending=True)
        print('RSEMArate參考資料50排序完成')
        rs_df['RS EMA50rate'] = scaler.fit_transform(np.array(rs_df['RS EMA50rate']).reshape(-1, 1))
        print('RSEMArate參考資料50已完成')
    except Exception as e:
        print(e)
        print('RSEMArate參考資料50失敗')
    try:
        rs_df['RS EMA20rate'] = rs_df['RS EMA20rate'].rank(method='dense', ascending=True)
        print('RSEMArate參考資料20排序完成')
        rs_df['RS EMA20rate'] = scaler.fit_transform(np.array(rs_df['RS EMA20rate']).reshape(-1, 1))
        print('RSEMArate參考資料20已完成')
    except Exception as e:
        print(e)
        print('RSEMArate參考資料20失敗')
    return rs_df
# 每日更新RS rate到每個csv
def daily_RS_update(timestart, rs_df):
    for i, id in enumerate(stock_num):
        name = id
        address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + str(id) + '.csv'
        try:
            stock = pd.read_csv(address, index_col = "Date",parse_dates = ["Date"])
        except Exception as e:
            print(e)
            print(name, 'RS rate更新失敗')
        try:
            id = int(id.split('.')[0])
            if timestart in stock.index:
                stock.loc[timestart, ['RS 20rate', 'RS 50rate', 'RS 250rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = rs_df.loc[id, ['RS 20rate', 'RS 50rate', 'RS 250rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']]
                stock.to_csv(address,encoding='utf-8-sig')
                print(name, 'RS rate更新成功')
            else:
                print(name, '沒有RS資料')
        except Exception as e:
            print(e)
            print(name, 'RS rate更新失敗')
# 更新RS MAX
def update_RS_MAX(stock_id):
    address = r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\history_data\\' + stock_id + ".csv"   
    try:
        if  os.path.isfile(address):
            df_new = pd.read_csv(address,index_col = "Date",parse_dates = ["Date"])
            try:
                df_new['ID'] = stock_id.split('.')[0]
            except Exception as e:
                print(e)
                print(stock_id,1)
            df_new['RS 250MA 10MAX'] = talib.MAX(df_new['RS 250rate'], 10)
            df_new['RS 50MA 10MAX'] = talib.MAX(df_new['RS 50rate'], 10)
            df_new['RS 20MA 10MAX'] = talib.MAX(df_new['RS 20rate'], 10)
            df_new['RS 250MA is 10MAX'] = df_new['RS 250rate'].round(1)>=df_new['RS 250MA 10MAX'].round(1)
            df_new['RS 50MA is 10MAX'] = df_new['RS 50rate'].round(1)>=df_new['RS 50MA 10MAX'].round(1)
            df_new['RS 20MA is 10MAX'] = df_new['RS 20rate'].round(1)>=df_new['RS 20MA 10MAX'].round(1)
            df_new['RS 250EMA 10MAX'] = talib.MAX(df_new['RS EMA250rate'], 10)
            df_new['RS 50EMA 10MAX'] = talib.MAX(df_new['RS EMA50rate'], 10)
            df_new['RS 20EMA 10MAX'] = talib.MAX(df_new['RS EMA20rate'], 10)
            df_new['RS 250EMA is 10MAX'] = df_new['RS EMA250rate'].round(1)>=df_new['RS 250EMA 10MAX'].round(1)
            df_new['RS 50EMA is 10MAX'] = df_new['RS EMA50rate'].round(1)>=df_new['RS 50EMA 10MAX'].round(1)
            df_new['RS 20EMA is 10MAX'] = df_new['RS EMA20rate'].round(1)>=df_new['RS 20EMA 10MAX'].round(1)
            df_new.to_csv(address,encoding='utf-8-sig')
            # print("指標已更新到最新資料")
    except Exception as err:
        print(err)
        print(stock_id + '指標更新失敗')
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
taiwan_0050 = pd.Series(['0050.TW'], index=[0])
taiwan_weighted = pd.Series(['^TWII'], index=[0])
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concat全部股票代號
stock_num = pd.concat([taiwan_weighted, taiwan_0050, stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(stock_num)
# print(type(allstock_info['ID'].values[0]))
# 每日更新csv
delay = 1
today=datetime.date.today() - datetime.timedelta(days=delay)
tomorrow = today + datetime.timedelta(days=1)
time_start = str(today)
time_end = str(tomorrow)
for i in stock_num :   
    # 更新價量
    try:
        stock_data(i, time_start,time_end)
        # print(i + "successful")
    except Exception as e:
        try:
            stock_data(i, time_start,time_end)
            # print(i + "successful")
        except Exception as e:
            print(i + "price fail")
            print(e)
    # 更新指標
    try:
        indicator(i)
        # print(i + "successful")
    except Exception as e:
        try:
            indicator(i)
            # print(i + "successful")
        except Exception as e:
            print(i + "indicator fail")
            print(e)
print('指標更新完成')
# 準備RS rate資料
timestart = datetime.datetime.strptime(str(datetime.date.today()+datetime.timedelta(days=-0)) , '%Y-%m-%d' )
rs_df = update_rs_rate(timestart)
print('rs rate更新完成')
rs_df = rs_rate_dataset(rs_df)
# 將RS rate資料匯入每個csv
daily_RS_update(timestart, rs_df)
for i in stock_num :   
    # 更新RS MAX
    try:
        update_RS_MAX(i)
        # print(i + "successful")
    except Exception as e:
        try:
            update_RS_MAX(i)
            # print(i + "successful")
        except Exception as e:
            print(i + "fail")
            print(e)